
# データ操作

:label: `sec_ndarray`

何かを成し遂げるには、データを保存して操作する何らかの方法が必要です。一般に、データに関して行う必要がある重要なことが 2 つあります。(i) データを取得する。 (ii) それらがコンピュータ内に入ったら処理します。データを取得しても、それを保存する方法がなければ意味がありません。まず、 $n$ 次元の配列 ( *tensor*とも呼ばれます) を使ってみましょう。 NumPy 科学計算パッケージをすでに知っている場合、これは簡単です。すべての最新の深層学習フレームワークでは、 *tensor クラス*(MXNet の`ndarray` 、PyTorch および TensorFlow の`Tensor` ) は NumPy の`ndarray`に似ていますが、いくつかのキラー機能が追加されています。まず、テンソル クラスは自動微分をサポートしています。第 2 に、NumPy は CPU 上でのみ実行されるのに対し、GPU を活用して数値計算を高速化します。これらの特性により、ニューラル ネットワークのコーディングが容易になり、実行も高速になります。

## 入門



(**まず、PyTorch ライブラリをインポートします。パッケージ名は`torch`であることに注意してください。** )


In [1]:
import torch


[**テンソルは、数値の (おそらく多次元の) 配列を表します。** ] 軸が 1 つあるテンソルは、*ベクトル*と呼ばれます。 2 つの軸を持つテンソルは*行列*と呼ばれます。 $k &gt; 2$ 軸では、特殊な名前を削除し、オブジェクトを $k^\mathrm{th}$*オーダー テンソル*として参照します。



PyTorch は、値が事前設定された新しいテンソルを作成するためのさまざまな関数を提供します。たとえば、 `arange(n)`を呼び出すと、0 (含まれる) で始まり`n` (含まれない) で終わる等間隔の値のベクトルを作成できます。デフォルトでは、間隔サイズは $1$ です。特に指定がない限り、新しいテンソルはメイン メモリに保存され、CPU ベースの計算用に指定されます。


In [2]:
x = torch.arange(12, dtype=torch.float32)
x

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])


これらの値のそれぞれは、テンソルの*要素*と呼ばれます。テンソル`x`には 12 個の要素が含まれます。 `numel`メソッドを使用して、テンソル内の要素の総数を検査できます。


In [3]:
x.numel()

12


(** `shape`属性を検査することで、テンソルの *形状***) (各軸に沿った長さ) にアクセスできます。ここではベクトルを扱っているため、 `shape`は要素が 1 つだけ含まれており、サイズも同じです。


In [4]:
x.shape

torch.Size([12])


`reshape`呼び出すことで、[**テンソルのサイズや値を変更せずにテンソルの形状を変更する**]ことができます。たとえば、形状 (12,) のベクトル`x`形状 (3, 4) の行列`X`に変換できます。この新しいテンソルはすべての要素を保持しますが、それらを行列に再構成します。ベクトルの要素は一度に 1 行ずつレイアウトされるため、 `x[3] == X[0, 3]`なることに注意してください。


In [5]:
X = x.reshape(3, 4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])


形状を`reshape`すべての形状コンポーネントを指定するのは冗長であることに注意してください。テンソルのサイズはすでにわかっているため、残りを考慮して形状の 1 つのコンポーネントを計算することができます。たとえば、サイズ $n$ とターゲット形状 ($h$, $w$) のテンソルが与えられた場合、$w = n/h$ であることがわかります。形状の 1 つのコンポーネントを自動的に推論するには、自動的に推論される形状コンポーネントに`-1`を設定します。この場合、 `x.reshape(3, 4)`を呼び出す代わりに、 `x.reshape(-1, 4)`または`x.reshape(3, -1)`を同等に呼び出すことができます。

実践者は多くの場合、すべて 0 または 1 を含むように初期化されたテンソルを操作する必要があります。 [**すべての要素がゼロに設定されたテンソル**] (~~または 1~~) と (2, 3, 4) の形状を、関数`zeros`を使用して構築できます。


In [6]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


同様に、 `ones`を呼び出すことで、すべて 1 のテンソルを作成できます。


In [7]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


私たちは、与えられた確率分布から[**各要素をランダムに(そして独立して)サンプリングする**]ことを望むことがよくあります。たとえば、ニューラル ネットワークのパラメーターはランダムに初期化されることがよくあります。次のスニペットは、平均 0、標準偏差 1 の標準ガウス (正規) 分布から抽出された要素を含むテンソルを作成します。


In [8]:
torch.randn(3, 4)

tensor([[ 1.4251, -1.4341,  0.2826, -0.4915],
        [ 0.1799, -1.1769,  2.3581, -0.1923],
        [ 0.8576, -0.0719,  1.4172, -1.3151]])


最後に、数値リテラルを含む (おそらくネストされた) Python リストを提供することにより、[**各要素の正確な値を提供する**] ことでテンソルを構築できます。ここでは、リストのリストを使用して行列を構築します。最も外側のリストは軸 0 に対応し、内側のリストは軸 1 に対応します。


In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])


## インデックス作成とスライス

Python リストと同様に、(0 から始まる) インデックスを付けることでテンソル要素にアクセスできます。リストの末尾からの相対的な位置に基づいて要素にアクセスするには、負のインデックスを使用できます。最後に、スライス (例: `X[start:stop]` ) を介してインデックスの全範囲にアクセスできます。この場合、戻り値には最初のインデックス ( `start` ) が含まれます*が、最後のインデックス*( `stop` ) は含まれません。最後に、 $k^\mathrm{th}$ 次数テンソルにインデックス (またはスライス) が 1 つだけ指定されている場合、それは軸 0 に沿って適用されます。 したがって、次のコードでは、 [ **`[-1]`最後の行を選択し、 `[1:3]`は 2 行目と 3 行目を選択します**。


In [10]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))


読み取りだけでなく、(**インデックスを指定して行列の要素を書き込むこともできます。** )


In [11]:
X[1, 2] = 17
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5., 17.,  7.],
        [ 8.,  9., 10., 11.]])


[**で複数の要素に同じ値を割り当てたい場合は、代入操作の左側にインデックスを適用します。** ] たとえば、 `[:2, :]`最初と 2 行目にアクセスします。ここで`:`は軸 1 (列) に沿ったすべての要素を取得します。行列のインデックス付けについて説明しましたが、これはベクトルや 2 次元を超えるテンソルにも機能します。


In [12]:
X[:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])


## オペレーション

テンソルの構築方法とその要素の読み取りおよび書き込み方法がわかったので、さまざまな数学的演算を使用してテンソルを操作し始めることができます。最も便利なツールの 1 つは*要素ごとの*操作です。これらは、標準のスカラー演算をテンソルの各要素に適用します。 2 つのテンソルを入力として受け取る関数の場合、要素ごとの演算では、対応する要素の各ペアに標準の二項演算子が適用されます。スカラーからスカラーにマップする任意の関数から要素ごとの関数を作成できます。

数学的表記法では、このような*単項*スカラー演算子 (1 つの入力を取る) を $f: \mathbb{R} \rightarrow \mathbb{R}$ という記号で表します。これは、関数が任意の実数を他の実数にマッピングすることを意味します。 $e^x$ のような単項演算子を含むほとんどの標準演算子は要素ごとに適用できます。


In [13]:
torch.exp(x)

tensor([162754.7969, 162754.7969, 162754.7969, 162754.7969, 162754.7969,
        162754.7969, 162754.7969, 162754.7969,   2980.9580,   8103.0840,
         22026.4648,  59874.1406])


同様に、署名 $f: \mathbb{R}, \mathbb{R} \rightarrow \mathbb{R}$ を介して実数のペアを (単一の) 実数にマッピングする二項*スカラー*演算子を示します。*同じ形状の*2 つのベクトル $\mathbf{u}$ と $\mathbf{v}$ 、および二項演算子 $f$ が与えられると、ベクトル $\mathbf{c} = F(\mathbf{u },\mathbf{v})$ を設定することにより、すべての $i$ に対して $c_i \gets f(u_i, v_i)$ を設定します。ここで、$c_i、u_i$、および $v_i$ は $i^\mathrm{th}$ ですベクトル $\mathbf{c}、\mathbf{u}$、および $\mathbf{v}$ の要素。ここでは、スカラー関数を要素ごとのベクトル演算に*リフトする*ことによって、ベクトル値 $F: \mathbb{R}^d, \mathbb{R}^d \rightarrow \mathbb{R}^d$ を生成しました。加算 ( `+` )、減算 ( `-` )、乗算 ( `*` )、除算 ( `/` )、およびべき乗 ( `**` ) の共通標準算術演算子はすべて、任意の形状の同一形状のテンソルに対する要素ごとの演算に*リフトアップ*されました。


In [14]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))


要素ごとの計算に加えて、ドット積や行列乗算などの線形代数演算も実行できます。これらについては、 :numref: `sec_linear-algebra`ですぐに詳しく説明します。

また、複数のテンソルを [***連結**し、端から端まで積み重ねて、より大きなテンソルを形成する] こともできます。テンソルのリストを提供し、どの軸に沿って連結するかをシステムに指示するだけです。以下の例は、2 つの行列を行 (軸 0) と列 (軸 1) に沿って連結すると何が起こるかを示しています。最初の出力の axis-0 の長さ ($6$) は、2 つの入力テンソルの axis-0 の長さ ($3 + 3$) の合計であることがわかります。一方、2 番目の出力の axis-1 の長さ ($8$) は、2 つの入力テンソルの axis-1 の長さ ($4 + 4$) の合計です。


In [15]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))


場合によっては、[***論理ステートメント*を介してバイナリ テンソルを構築したいことがあります。** ] `X == Y`例に挙げます。各位置`i, j`について、 `X[i, j]`と`Y[i, j]`が等しい場合、結果の対応するエントリは値`1`をとり、それ以外の場合は値`0`をとります。


In [16]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])


[**テンソル内のすべての要素を合計すると**] 要素が 1 つだけのテンソルが得られます。


In [17]:
X.sum()

tensor(66.)


## 放送

:label: `subsec_broadcasting`

ここまでで、同じ形状の 2 つのテンソルに対して要素ごとの二項演算を実行する方法を理解しました。特定の条件下では、形状が異なっていても、[***ブロードキャスト メカニズム*を呼び出すことによって要素ごとのバイナリ操作を実行できます。** ] ブロードキャストは、次の 2 段階の手順に従って機能します。(i) 長さ 1 の軸に沿って要素をコピーすることによって一方または両方の配列を拡張し、この変換後に 2 つのテンソルが同じ形状になるようにします。 (ii) 結果の配列に対して要素ごとの演算を実行します。


In [18]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))


`a`と`b`はそれぞれ $3\times1$ と $1\times2$ 行列なので、それらの形状は一致しません。ブロードキャストでは、要素ごとに加算する前に、行列`a`を列に沿って複製し、行列`b`行に沿って複製することにより、より大きな $3\times2$ 行列が生成されます。


In [19]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])


## メモリの節約

[**操作を実行すると、ホストの結果に新しいメモリが割り当てられる場合があります。** ] たとえば、 `Y = X + Y`と書くと、 `Y`がポイントしていたテンソルを逆参照し、代わりに新しく割り当てられたメモリを`Y`にポイントします。 Python の`id()`関数を使用してこの問題を実証できます。これにより、メモリ内の参照オブジェクトの正確なアドレスが得られます。 `Y = Y + X`を実行すると、 `id(Y)`別の場所を指すことに注意してください。これは、Python が最初に`Y + X`を評価し、結果に新しいメモリを割り当ててから、メモリ内のこの新しい場所を`Y`に指すためです。


In [20]:
before = id(Y)
Y = Y + X
id(Y) == before

False


これは 2 つの理由から望ましくない可能性があります。まず、常に不必要にメモリを割り当て続ける必要はありません。機械学習では、多くの場合、数百メガバイトのパラメーターがあり、それらすべてが 1 秒間に複数回更新されます。可能な限り、これらの更新を*適切な場所で*実行したいと考えています。 2 番目に、複数の変数から同じパラメーターを指す可能性があります。適切な場所で更新しない場合は、メモリ リークが発生したり、古いパラメータを誤って参照したりしないように、これらの参照をすべて更新するように注意する必要があります。



幸いなことに、(**インプレース操作の実行**) は簡単です。スライス表記`Y[:] = &lt;expression&gt;`を使用して、演算の結果を以前に割り当てられた配列`Y`に割り当てることができます。この概念を説明するために、テンソル`Z`の値を初期化した後、 `zeros_like`を使用して`Y`と同じ形状になるように上書きします。


In [21]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 139763606871712
id(Z): 139763606871712



[ **`X`の値が後続の計算で再利用されない場合は、 `X[:] = X + Y`または`X += Y`を使用して、操作のメモリ オーバーヘッドを減らすこともできます。** 】


In [22]:
before = id(X)
X += Y
id(X) == before

True


## 他の Python オブジェクトへの変換



[ **NumPy テンソル ( `ndarray` ) への変換**]、またはその逆は簡単です。 torch Tensor と numpy 配列は基礎となるメモリを共有し、インプレース操作で一方を変更すると、もう一方も変更されます。


In [23]:
A = X.numpy()
B = torch.from_numpy(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)


(**サイズ 1 のテンソルを Python スカラーに変換する**) には、 `item`関数または Python の組み込み関数を呼び出すことができます。


In [24]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)


## まとめ

テンソル クラスは、ディープ ラーニング ライブラリにデータを保存および操作するためのメイン インターフェイスです。 Tensor は、構築ルーチンを含むさまざまな機能を提供します。インデックス付けとスライス。基本的な数学演算。放送;メモリ効率の高い割り当て。他の Python オブジェクトとの間の変換。

## 演習
1. このセクションのコードを実行します。条件ステートメント`X == Y` `X &lt; Y`または`X &gt; Y`に変更して、どのような種類のテンソルが得られるかを確認してください。
1. ブロードキャスト メカニズムの要素によって動作する 2 つのテンソルを、他の形状 (たとえば 3 次元テンソル) に置き換えます。結果は期待どおりですか?



[ディスカッション](https://discuss.d2l.ai/t/27)
